# Team Ranking based on Colley's method
Resources used:
- https://www.colleyrankings.com/matrate.pdf ($\textbf{Colley's own description of the method}$ - reliable but bad notation and lengthy)
- https://www.dcs.bbk.ac.uk/~ale/dsta+dsat/dsta+dsat-3/lm-ch3-colley.pdf (watch out for small errors)

###  Method Description
One can imagine a simple ranking system based on the winning percentage (of team $i$)

$$r_i = \frac{w_i}{t_i}$$

the teams. Colley proposed a simple modification which
- takes into account the opponent players strength
- makes rankings "zero sum" (teams steal "ranking score", rankings does not really sum to 0, but $n_{\text{teams}}/2$)
- initially more stabler (a team who loses once does not have 0 ranking, but moves more slowly away from the teams' equilibrium)

by instead using the (Laplace modified) winning percentage

$$r_i = \frac{w_i+1}{t_i+2}.$$

Consider that we may write

$$ w_i = \frac{w_i-l_i}{2} + \frac{w_i+l_i}{2} = \frac{w_i-l_i}{2} + \frac{t_i}{2}$$

where we may estimate the $\frac{t_i}{2}$ as

$$ \sum_{i=1}^{t_i} \frac{1}{2} \approx \sum_{j\in \mathcal{O}_i} n_{i,j} r_j $$

where $\mathcal{O}_i$ is the set of opponents of team i and $n_{i,j}$ is the number of matches between $i$ and $j$ (note that the approximation is justified by the fact that the equilibrium of ratings is $\frac{1}{2}$, but the approximation may be bad if there are multiple teams who only wins and loses - should not be a problem in fair match ups or sports where domination is unusual). We may then write

$$ r_i \approx \frac{\frac{w_i-l_i}{2} - \sum_{j\in \mathcal{O}_i} n_{i,j} r_j +1}{t_i+2} $$

where, using this approximation, we now have that the rating of one team is dependent on the other teams. Further, we see that the equation is a system of $n_{\text{teams}}$ equations with $n_{\text{teams}}$ variables as $i \in \{ 1,2,...,n_{\text{teams}} \}$:

$$ (t_i + 2)r_i - \sum_{j\in \mathcal{O}_i} n_{i,j} r_j = \frac{w_i-l_i}{2} + 1 $$

or, in matrix form

$$ Cr = b $$

where $b_i = \frac{w_i-l_i}{2} +1 $ and the diagonal elements of $C$ are $C_{i,i}=t_i+2$ and the off-diagonal elements are $C_{i,j} = -n_{i,j}$ for $i \neq j$.

Note that $C$ is symmetric, real and positive definite (discussed by Colley) hence $C$ can be subjected to Cholesky decomposition.

### Applying the method
We will apply the method to the soccer league 'Allsvenskan' results of 2021 to rank the teams. We can then compare the rankings to to the offical results found at https://www.svenskfotboll.se/serier-cuper/tabell-och-resultat/allsvenskan-2021/88307/.

Of course, soccer has the win/draw/lose structure. We will see a draw as half a win and half a loss in our implementation. Another solution could be to filter out draws.

In [24]:
import pandas as pd
import numpy as np
# Import train & test data 
data_temp = pd.read_csv('C:/Users/xsoni/Desktop/allsvenskan.txt')

# Date reformating to be able to easily select dates and then pick 2021
data_temp["Date"]=(data_temp["Date"].str.replace("-","")).astype(int)
df = data_temp[20210000<data_temp['Date']]
df = df[df['Date']<20220000]

# We define the "winner" columns as =1 if the home team won, 0 if draw and -1 if the away team won
def winner(row):
    if row['FTHG']>row['FTAG']:
        return 'H'
    elif row['FTHG']==row['FTAG']:
        return 'D'
    else:
        return 'A'

# Add result in terms of who won/draw
df['Result'] = df.apply (lambda row: winner(row), axis=1)

# Pick only what we need: H_team, A_team, Date (in case) and Result
allsvenskan2021 = df[['Date', 'H_team', 'A_team', 'Result']]

# Helsingborg only has 2 matches, we remove matches with Helsingborg such that all teams have 30 matches
allsvenskan2021 = allsvenskan2021[allsvenskan2021['H_team'] != 'Helsingborg']
allsvenskan2021 = allsvenskan2021[allsvenskan2021['A_team'] != 'Helsingborg']

allsvenskan2021.head()

,Date,H_team,A_team,Result
1922,20210410,MalmoFF,Hammarby,H
1923,20210410,Orebro,Goteborg,D
1924,20210411,Mjallby,Varberg,D
1925,20210411,Halmstad,Hacken,H
1926,20210411,Norrkoping,Sirius,D


We will need a couple of functions to fill the Colley matrix and the 'b' vector. Recall that $b_i = \frac{w_i-l_i}{2} +1$, $C_{i,i}=t_i+2$ and $C_{i,j} = -n_{i,j}$ for $i \neq j$. Hence we implement functions to answer the following questions:
- How many matches has team i played?
- Out of those matches, how many are wins and how many are losses? How many were played against team j?

In [60]:
def matches(team):
    return len(allsvenskan2021[(allsvenskan2021['H_team'] == team) | (allsvenskan2021['A_team'] == team)])

def wins(team):
    temp = allsvenskan2021[(allsvenskan2021['H_team'] == team) | (allsvenskan2021['A_team'] == team)]
    wins = temp[((temp['H_team'] == team) & (temp['Result'] == 'H')) | ((temp['A_team'] == team) & (temp['Result'] == 'A'))]
    draws = temp[( (temp['H_team'] == team) | (temp['A_team'] == team) ) & (temp['Result'] == 'D')]
    return len(wins)+1/2*len(draws)

def losses(team):
    return matches(team)-wins(team)

def matchups(team1, team2):
    temp = allsvenskan2021[(allsvenskan2021['H_team'] == team1) | (allsvenskan2021['A_team'] == team1)]
    return len(temp[(temp['H_team'] == team2) | (temp['A_team'] == team2)])

In [125]:
# Make a set of teams
A_teams = allsvenskan2021['A_team'].unique()
H_teams = allsvenskan2021['H_team'].unique()
teams = list(set(np.concatenate((A_teams, H_teams), axis=0)))
n_teams = len(teams)


# Initialize Colley matrix C (vectorized way of doing it would be a more efficient way of initializing)
C = np.zeros([n_teams, n_teams])
for i in range(n_teams):
    for j in range(n_teams):
        if i == j:
            C[i][j] = matches(teams[i])+2
        if i != j:
            C[i][j] = -matchups(teams[i], teams[j])
            
# Initialize vector b
b = np.zeros(n_teams)
i = 0
for team in teams:
    b[i] = (wins(team)-losses(team))/2 + 1
    i=i+1
    
# We solve the Colley equation for ratings using a simple solver (small matrix anyways, no need for Cholesky decomp.)
r = np.linalg.solve(C, b)

In [143]:
team_ratings = list(zip(teams, list(r)))
team_ratings.sort(key = lambda x: x[1])
team_ratings.reverse()
team_ratings

[('MalmoFF', 0.6764705882352935),
 ('AIK', 0.6617647058823523),
 ('Djurgarden', 0.6470588235294112),
 ('Hammarby', 0.6176470588235288),
 ('Elfsborg', 0.6176470588235287),
 ('Kalmar', 0.558823529411764),
 ('Norrkoping', 0.5147058823529406),
 ('Goteborg', 0.4999999999999992),
 ('Mjallby', 0.48529411764705815),
 ('Varberg', 0.4705882352941169),
 ('Hacken', 0.4558823529411758),
 ('Sirius', 0.4558823529411758),
 ('Halmstad', 0.4411764705882346),
 ('Degerfors', 0.41176470588235226),
 ('Orebro', 0.2647058823529405),
 ('Ostersund', 0.22058823529411697)]

As compared to the true rating found here: https://www.svenskfotboll.se/serier-cuper/tabell-och-resultat/allsvenskan-2021/88307/ we see that the algorithm gives a reasonable output. Further investigations may be added at a later point.